In [1]:
%cd ..

/home/max/process/dist/app_noisemon


In [2]:
import pandas as pd
from tqdm import tqdm
import json

from pathlib import Path

In [3]:
from noisemon.database.database import SessionLocal

from noisemon.models.entity import EntityModel, get_insert_many_statement
from noisemon.models.document import DocumentData, DocumentModel, DocumentOrigin
from noisemon.models.document import dataclass_to_model as document_dataclass_to_model
from noisemon.models.mention import *


In [4]:
trex_dir = Path("./assets/entity_linking/TREx/").resolve()

In [5]:
for f in trex_dir.iterdir():
    print(f)
    break

/home/max/process/dist/app_noisemon/assets/entity_linking/TREx/re-nlg_2370000-2380000.json


In [6]:
db = SessionLocal()
with db.begin():
    buffer = []
    for idx, row in tqdm(item_df.iterrows(), total=len(item_df)):
        new_entity = EntityModel(
            qid=id_to_qid(idx),
            name=row.en_label,
            type=None,
        )
        buffer.append(new_entity)
        
        if idx % 5000 == 1:
            statement = get_insert_many_statement(buffer)
            db.execute(statement)
            buffer = []

    else:
        statement = get_insert_many_statement(buffer)
        db.execute(statement)
       
db.commit()
db.close()

NameError: name 'item_df' is not defined

In [13]:
files = list(trex_dir.glob("*.json"))
db = SessionLocal()

for file in tqdm(files):
    with db.begin():
        with open(file, "r") as fin:
            content: list[dict] = json.load(fin)
            for chunk in content:
                # document origin
                origin = DocumentOrigin(
                    resource=f"WikiPedia article [{chunk['title']}][:intro]",
                    from_process="kb_population"
                )
                # document
                document = DocumentData(
                    origin=origin,
                    text=chunk["text"],
                    raw_text=chunk["text"],
                )
                new_document = document_dataclass_to_model(document)
                db.add(new_document)
                db.flush()
                
                for annotation in chunk["entities"]:
                    if annotation["annotator"] != "Wikidata_Spotlight_Entity_Linker": continue
                    span_start, span_end = annotation["boundaries"]
                    span = chunk["text"][span_start:span_end]
                    qid = annotation["uri"]

                    new_mention = MentionModel(
                        origin_id=new_document.id,
                        span=span,
                        span_start=span_start,
                        span_end=span_end,
                        entity_qid=qid, 
                        vector_index=0
                    )
                    db.add(new_mention)

                db.flush()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 465/465 [11:25:01<00:00, 88.39s/it]


In [14]:
db.commit()

In [15]:
db.close()

In [14]:
annotators = set()

In [17]:
for c in chunk:
    for e in c["entities"]:
        annotators.add(e["annotator"])
    

In [18]:
annotators

{'Date_Linker',
 'Simple_Coreference',
 'Wikidata_Property_Linker',
 'Wikidata_Spotlight_Entity_Linker'}

In [26]:
chunk

{'docid': 'http://www.wikidata.org/entity/Q3446429',
 'title': 'Ruairí Ó Brádaigh',
 'text': 'Ruairí Ó Brádaigh ([ˈɾˠuəɾʲiː oː bˠɾˠaːd̪ˠiː], born Peter Roger Casement Brady; 2 October 1932 – 5 June 2013) was an Irish republican paramilitary and political leader. He was Chief of Staff of the Irish Republican Army (IRA) from 1958 to 1959 and again from 1960 to 1962, president of Sinn Féin from 1970 to 1983, and president of Republican Sinn Féin from 1987 to 2009.',
 'uri': 'http://www.wikidata.org/entity/Q3446429',
 'entities': [{'boundaries': [52, 57],
   'surfaceform': 'Peter',
   'uri': 'http://www.wikidata.org/entity/Q33923',
   'annotator': 'Wikidata_Spotlight_Entity_Linker'},
  {'boundaries': [58, 72],
   'surfaceform': 'Roger Casement',
   'uri': 'http://www.wikidata.org/entity/Q311966',
   'annotator': 'Wikidata_Spotlight_Entity_Linker'},
  {'boundaries': [117, 122],
   'surfaceform': 'Irish',
   'uri': 'http://www.wikidata.org/entity/Q9142',
   'annotator': 'Wikidata_Spotlight_E

In [5]:
with open("/home/max/process/dist/app_noisemon/assets/entity_linking/TREx/re-nlg_2370000-2380000.json", "r") as fin:
    chunk = json.loads(line)
            page_id = chunk["page_id"]
            page_title = page_df.iloc[page_id].title
            origin = DocumentOrigin(resource=f"WikiPedia Page [{page_title}]", from_process="kb_population")

            for section in chunk.get("sections", []):
                origin = DocumentOrigin(
                    resource=f"WikiPedia Page [{page_title}][section: {section['name']}]", 
                    from_process="kb_population"
                )
                document = DocumentData(
                    origin=origin,
                    text=section["text"],
                    raw_text=section["text"],
                )
                new_document = document_dataclass_to_model(document)
                db.add(new_document)
                db.commit()
                
                for span_start, span_end, span, title, qid in parse_section(section, page_df):
                    new_mention = MentionModel(
                        origin=new_document,
                        span=span,
                        span_start=span_start,
                        span_end=span_end,
                        entity_qid=id_to_qid(qid),
                        
                    )
                

In [9]:
list(chunk.keys())

['page_id', 'sections']

In [10]:
list(chunk["sections"][0].keys())

['name', 'text', 'link_lengths', 'link_offsets', 'target_page_ids']

In [12]:
section = chunk["sections"][0]

In [14]:
section["text"][16:16+18]

'anti-authoritarian'

In [27]:
parse_section(section, page_df)

anti-authoritarian Anti-authoritarianism 1030234
political Political philosophy 179805
social philosophy Social philosophy 180592
hierarchies Hierarchy 188619
self-managed Organizational Self-management 15981562
self-governed Self-governance 417633
cooperative Cooperative 4539
stateless societies Stateless society 1708603
free associations Free association (Marxism and anarchism) 5500187
state Sovereign state 3624078
far-left Far-left politics 1129409
political spectrum Political spectrum 210918
economics Anarchist economics 4751666
legal philosophy Anarchist law 2738779
communism Anarcho-communism 188670
collectivism Collectivist anarchism 223178
syndicalism Anarcho-syndicalism 188993
mutualism Mutualism (economic theory) 844212
participatory economics Participatory economics 650953
anarchist types and traditions History of anarchism 2746978
Anarchist schools of thought Anarchist schools of thought 1278791
individualism Individualism 171995
collectivism Collectivism 237789
social Soci

In [15]:
section

{'name': 'Introduction',
 'text': "Anarchism is an anti-authoritarian political and social philosophy that rejects hierarchies deemed unjust and advocates their replacement with self-managed, self-governed societies based on voluntary, cooperative institutions. These institutions are often described as stateless societies, although several authors have defined them more specifically as distinct institutions based on non-hierarchical or free associations. Anarchism's central disagreement with other ideologies is that it holds the state to be undesirable, unnecessary, and harmful. Anarchism is usually placed on the far-left of the political spectrum, and much of its economics and legal philosophy reflect anti-authoritarian interpretations of communism, collectivism, syndicalism, mutualism, or participatory economics. As anarchism does not offer a fixed body of doctrine from a single particular worldview, many anarchist types and traditions exist and varieties of anarchy diverge widely. A

In [18]:
page_df[page_df.page_id == 867979]

,page_id,item_id,title,views
260552,867979,1030234,Anti-authoritarianism,1914
